In [44]:
from utils.utils import connection
import pandas as pd
from stockstats import StockDataFrame
# import numpy as np
# from time import time
# from datetime import date, datetime

In [3]:
conn = connection()

candles = pd.read_sql("""
    SELECT * FROM candles 
    WHERE datetime >= NOW() - INTERVAL '25 years'
    ORDER BY datetime DESC
""", conn)

In [4]:
candles.head(5)

,id,ticker,datetime,open,high,low,close,volume,is_complete,created_at
0,145676,ASTR,2025-11-26,293.8500,295.000,290.75,291.350,203508,False,2025-11-26 17:33:15.497990
1,145675,IRAO,2025-11-26,2.9015,2.915,2.88,2.893,475270,False,2025-11-26 17:33:14.919387
2,145674,MOEX,2025-11-26,175.6200,175.620,173.10,173.270,338790,False,2025-11-26 17:33:14.347330
3,145673,VTBR,2025-11-26,72.0500,72.100,71.46,71.730,26533256,False,2025-11-26 17:33:13.772946
4,145672,RASP,2025-11-26,172.8500,175.350,170.10,172.100,123963,False,2025-11-26 17:33:13.196495


In [34]:
def data_from_ticker(ticker: str, left_date: str, right_date: str, conn) -> pd.DataFrame:
    shares = pd.read_sql(f"""
        SELECT * FROM candles
        WHERE ticker = '{ticker}'
        AND datetime >= '{left_date}' AND datetime <= '{right_date}'
        ORDER BY datetime DESC
    """, conn)
    
    shares = shares.drop_duplicates()
                
    shares['prev_close'] = shares['close'].shift(1)
    shares['open_to_prev_close'] = shares['open'] / shares['prev_close'] - 1
    shares["target"] = shares["close"]
    shares['delta_open_close'] = shares['open'] - shares['close']
    shares['delta_open_close_pct'] = (shares['open'] - shares['close']) / shares['close']
    
    stock = StockDataFrame.retype(shares)
    
    shares['macd'] = stock['macd']
    shares['rsi_14'] = stock['rsi_14']
    shares['boll_ub'] = stock['boll_ub']
    shares['boll_lb'] = stock['boll_lb']
    
    shares = shares.drop(columns=['open', 'prev_close', 'close'])
                
    for col in shares.columns:
        shares[col] = shares[col].ffill()
        
    shares = shares.dropna().reset_index(drop=True)
    
    return shares

In [35]:
df = data_from_ticker("MOEX", "2000-11-21", "2025-11-25", conn)

In [36]:
df.head()

,id,ticker,datetime,high,low,volume,is_complete,created_at,open_to_prev_close,target,delta_open_close,delta_open_close_pct,macd,rsi_14,boll_ub,boll_lb
0,145546,MOEX,2025-11-24,178.54,172.40,1226833,False,2025-11-24 17:31:59.072150,-0.015014,172.64,0.56,0.003244,-0.071795,0.0,178.765483,169.714517
1,145482,MOEX,2025-11-21,172.80,170.89,58329,False,2025-11-21 05:22:44.528054,-0.000811,171.86,0.64,0.003724,-0.115669,0.0,177.664786,169.228547
2,145418,MOEX,2025-11-20,169.48,168.60,22740,False,2025-11-20 05:22:07.523547,-0.013848,168.98,0.50,0.002959,-0.236770,0.0,177.970284,166.689716
3,145354,MOEX,2025-11-19,163.07,161.61,21345,False,2025-11-19 05:21:30.034153,-0.042431,162.80,-0.99,-0.006081,-0.543506,0.0,180.248272,160.599728
4,145290,MOEX,2025-11-18,160.49,159.29,8057,False,2025-11-18 04:42:18.127945,-0.018366,160.38,-0.57,-0.003554,-0.821161,0.0,180.769471,156.730529


In [37]:
df.shape

(2598, 16)

In [38]:
df.dtypes

id                               int64
ticker                          object
datetime                datetime64[ns]
high                           float64
low                            float64
volume                           int64
is_complete                       bool
created_at              datetime64[ns]
open_to_prev_close             float64
target                         float64
delta_open_close               float64
delta_open_close_pct           float64
macd                           float64
rsi_14                         float64
boll_ub                        float64
boll_lb                        float64
dtype: object

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2598 entries, 0 to 2597
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    2598 non-null   int64         
 1   ticker                2598 non-null   object        
 2   datetime              2598 non-null   datetime64[ns]
 3   high                  2598 non-null   float64       
 4   low                   2598 non-null   float64       
 5   volume                2598 non-null   int64         
 6   is_complete           2598 non-null   bool          
 7   created_at            2598 non-null   datetime64[ns]
 8   open_to_prev_close    2598 non-null   float64       
 9   target                2598 non-null   float64       
 10  delta_open_close      2598 non-null   float64       
 11  delta_open_close_pct  2598 non-null   float64       
 12  macd                  2598 non-null   float64       
 13  rsi_14            

In [40]:
df.describe()

,id,datetime,high,low,volume,created_at,open_to_prev_close,target,delta_open_close,delta_open_close_pct,macd,rsi_14,boll_ub,boll_lb
count,2598.000000,2598,2598.000000,2598.000000,2.598000e+03,2598,2598.000000,2598.000000,2598.000000,2598.000000,2598.000000,2598.000000,2598.000000,2598.000000
mean,111732.789838,2020-12-15 05:53:04.295611904,138.175239,134.798533,7.846044e+05,2025-11-13 08:47:55.817950464,0.000388,136.534269,0.053676,0.000479,-0.195050,48.238501,144.707308,128.916130
min,110322.000000,2015-11-16 00:00:00,75.800000,69.700000,3.881000e+03,2025-11-13 08:18:03.472831,-0.156842,73.190000,-23.420000,-0.093883,-10.630372,0.000000,82.185843,67.863742
25%,110971.250000,2018-05-15 06:00:00,103.210000,100.230000,4.274010e+05,2025-11-13 08:18:03.472830976,-0.013053,101.550000,-1.040000,-0.008023,-1.731185,38.947075,108.418198,94.814949
50%,111620.500000,2020-12-08 12:00:00,119.985000,117.275000,6.583605e+05,2025-11-13 08:18:03.472830976,0.000708,118.750000,0.110000,0.000824,-0.334049,47.847626,127.361413,112.384231
75%,112269.750000,2023-07-30 06:00:00,174.787500,170.975000,9.684990e+05,2025-11-13 08:18:03.472830976,0.013263,172.810000,1.180000,0.009188,1.339084,58.023104,180.890240,165.807838
max,145546.000000,2025-11-24 00:00:00,252.680000,247.150000,7.715720e+06,2025-11-24 17:31:59.072150,0.242706,251.940000,13.620000,0.125459,12.062059,79.816293,258.952166,231.821180
std,2117.109248,NaN,43.781680,42.746047,5.887297e+05,NaN,0.024922,43.281439,2.383688,0.017058,2.790433,12.783634,45.551886,40.951880


In [41]:
df.describe(include=["object", "bool"])

,ticker,is_complete
count,2598,2598
unique,1,2
top,MOEX,True
freq,2598,2591


In [42]:
df.isna().sum()

id                      0
ticker                  0
datetime                0
high                    0
low                     0
volume                  0
is_complete             0
created_at              0
open_to_prev_close      0
target                  0
delta_open_close        0
delta_open_close_pct    0
macd                    0
rsi_14                  0
boll_ub                 0
boll_lb                 0
dtype: int64

In [43]:
df.duplicated().sum()

np.int64(0)